runs the same code as heat_1d_1d_example.ipynb but now with the new heat_nn implementation

In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from heat_combined import heat_nn
np.random.seed(238)
torch.manual_seed(301)

In [2]:
layers = [32, 64, 128, 128, 1]
#in this example: the output layer uses identity as activation func and all the hidden layers use tanh
activations = [torch.tanh]*(len(layers)-1) + [None]
dim=1
kappa = 0.1
u_0 = lambda x: torch.sin(torch.pi * x) + torch.sin(4 * torch.pi * x)
rhs = lambda x: torch.sin(torch.pi * x)
u_analytic = lambda x,t: (1 - 1 / (0.1 * np.pi**2)) * np.sin(np.pi * x) * np.exp(-np.pi**2 * 0.1 * t
                    ) + np.sin(4 * np.pi * x) * np.exp(- 16 * np.pi**2 * 0.1 * t
                    ) + 1 / (0.1 * np.pi**2) * np.sin(np.pi * x) 

pde_nn = heat_nn(layers, activations, dim, u_0, kappa, rhs)
pde_nn.set_analytic_solution(u_analytic)

N_colloc = 100
x = np.linspace(0, 1, N_colloc)
t = np.linspace(0, 0.2, N_colloc)

xs, ts = np.meshgrid(x, t)

xs = torch.tensor(xs, dtype=torch.float32).view(-1, 1)
ts = torch.tensor(ts, dtype=torch.float32).view(-1, 1)

x_colloc = torch.tensor(xs, dtype=torch.float32)
t_colloc = torch.tensor(ts, dtype=torch.float32)

"""
N = 1000
x_colloc= torch.rand(N,1)
t_colloc = 0.2*torch.rand(N,1)
"""
# gives relatively good results (compared to other parameters, still bad though)
pde_nn.train(x_colloc, t_colloc, lr=1e-2, weight_decay=0.0, epochs = 75, opt_time_scale =True, print_epochs=50)
# LBFGS needs approximately 100 epochs, 30 iterations for kappa = 1
# if kappa = 0.1, better choose more iterations, less epochs
pde_nn.train_lbfgs(x_colloc, t_colloc, lr=1, opt_time_scale = True, epochs=1, max_iter=100)

for t in [0.001, 0.005, 0.01, 0.05, 0.1, 0.2]:
    x_test = torch.linspace(0,1,100).view(-1,1)
    t_test = torch.tensor([[t]]*100)  # t=0.5
    u_pred = pde_nn.trial_solution(x_test, t_test).detach().numpy()


    #compare results with analytic solution

    x_np = x_test.numpy().flatten()  # convert to 1D array for plotting
    t_val = t
    u_analytic = (1 - 1 / (0.1 * np.pi**2)) * np.sin(np.pi * x_np) * np.exp(-np.pi**2 * 0.1 * t_val
                    ) + np.sin(4 * np.pi * x_np) * np.exp(- 16 * np.pi**2 * 0.1 * t_val
                    ) + 1 / (0.1 * np.pi**2) * np.sin(np.pi * x_np) 

    # Plot the results
    plt.figure(figsize=(8,5))
    plt.plot(x_np, u_pred, label='NN Prediction', linewidth=2)
    plt.plot(x_np, u_analytic, '--', label='Analytic Solution', linewidth=2)
    plt.xlabel('x')
    plt.ylabel(f'u(x,t={t})')
    plt.ylim(-0.75, 2.0)
    plt.title('Heat Equation: NN vs Analytic Solution')
    plt.legend()
    plt.grid(True)
    plt.show()

/tmp/ipykernel_32001/1979655208.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_colloc = torch.tensor(xs, dtype=torch.float32)
/tmp/ipykernel_32001/1979655208.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t_colloc = torch.tensor(ts, dtype=torch.float32)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)